# Modifiying Recipes

In [1]:
from tim_reasoning import RecipeTagger

### Tagger Example

In [2]:
ner_model_path = '/Users/rlopez/PTG/experiments/models/xlm-roberta-base-finetuned-recipe-all'
srl_model_path = '/Users/rlopez/PTG/experiments/models/structured-prediction-srl-bert.2020.12.15.tar.gz'
tagger = RecipeTagger(ner_model_path, srl_model_path)

/Users/rlopez/opt/anaconda3/envs/allennlp/lib/python3.7/site-packages/allennlp/tango/__init__.py:18: UserWarning: AllenNLP Tango is an experimental API and parts of it might change or disappear every time we release a new version.
  "AllenNLP Tango is an experimental API and parts of it might change or disappear "


In [3]:
tokens, tags = tagger.predict_entities('25 grams whole coffee beans.')
tagger.plot_entities(tokens, tags)
actions = tagger.extract_action_relations(tokens, tags)
print('Actions:', actions)
tokens, tags = tagger.predict_entities('Let the coffee drain completely into the mug before removing the dripper.')
tagger.plot_entities(tokens, tags)
actions = tagger.extract_action_relations(tokens, tags)
print('Actions:', actions)

Actions: []


Actions: [('Let', 'coffee'), ('drain', None), ('removing', None)]


### Loading Resources: Actions, Ingredients, Units and Quantities

In [4]:
import pandas as pd

actions_data = pd.read_csv('/Users/rlopez/PTG/experiments/datasets/epic_kitchens/EPIC_100_verb_classes.csv')
actions_data

,id,key,instances,category
0,0,take,"['collect-from', 'collect-into', 'draw', 'fetc...",retrieve
1,1,put,"['create', 'dose', 'lay', 'lay-down', 'lay-on'...",leave
2,2,wash,"['clean', 'clean-around', 'clean-from', 'clean...",clean
3,3,open,"['lever-open', 'open', 'open-in', 'open-on', '...",access
4,4,close,"['close', 'close-off', 'close-with', 'screw-on...",block
...,...,...,...,...
92,92,prepare,"['prepare', 'prepare-for']",manipulate
93,93,bake,['bake'],manipulate
94,94,mark,"['mark', 'mark-on']",manipulate
95,95,bend,['bend'],manipulate


In [5]:
import ast

def load_action_groups():
    action_groups = {}
    for index, row in  actions_data.iterrows():
        main_action = row['key']
        related_actions = ast.literal_eval(row['instances'])
        related_actions = [main_action] + related_actions
        for  action in related_actions:
            action_groups[ action] = [v for v in related_actions if v != action]
    
    return action_groups

action_groups = load_action_groups()
print('Found %d actions' % len(action_groups))

Found 974 actions


In [6]:
nyt_data = pd.read_csv('/Users/rlopez/PTG/experiments/datasets/nyt/nyt-ingredients-snapshot-2015.csv')
nyt_data

,index,input,name,qty,range_end,unit,comment
0,0,1 1/4 cups cooked and pureed fresh butternut s...,butternut squash,1.25,0.0,cup,"cooked and pureed fresh, or 1 10-ounce package..."
1,1,1 cup peeled and cooked fresh chestnuts (about...,chestnuts,1.00,0.0,cup,"peeled and cooked fresh (about 20), or 1 cup c..."
2,2,"1 medium-size onion, peeled and chopped",onion,1.00,0.0,NaN,"medium-size, peeled and chopped"
3,3,"2 stalks celery, chopped coarse",celery,2.00,0.0,stalk,chopped coarse
4,4,1 1/2 tablespoons vegetable oil,vegetable oil,1.50,0.0,tablespoon,NaN
...,...,...,...,...,...,...,...
179202,179202,3/4 oz. pineapple juice,pineapple juice,0.75,0.0,ounce,NaN
179203,179203,1 tsp. fresh lemon juice,lemon juice,1.00,0.0,teaspoon,fresh
179204,179204,Angostura bitters,Angostura bitters,0.00,0.0,NaN,NaN
179205,179205,Wedge of pineapple,pineapple,1.00,0.0,wedge,NaN


In [7]:
from fractions import Fraction
import math

def get_frequent_entities(entity, min_frequency=20):
    entity_counts = entity.value_counts()
    entity_counts = entity_counts[entity_counts > min_frequency]
    
    return entity_counts.index.tolist()

def create_fraction(number):
    if number < 1:
        fraction = str(Fraction(number).limit_denominator(10))
        return fraction
    else:
        decimal_part, int_part  = math.modf(number)
        if decimal_part == 0:
            return str(int(int_part))
        else:
            fraction = str(Fraction(decimal_part).limit_denominator(10))
            return '%d %s' % (int_part, fraction)
        
nyt_data['name'] = nyt_data['name'].str.lower()
nyt_data['unit'] = nyt_data['unit'].str.lower()

ingredients = get_frequent_entities(nyt_data['name'])
units = get_frequent_entities(nyt_data['unit'])
quantities = get_frequent_entities(nyt_data['qty'])
quantities = [create_fraction(x)  for x in quantities]

print('Found %d unique ingredients: %s...' % (len(ingredients), str(ingredients[:5])))
print('Found %d unique units: %s...' % (len(units), str(units[:5])))
print('Found %d unique quantities: %s...' % (len(quantities), str(quantities[:5])))

corpus_recipe_entities = {'ingredients': ingredients, 'units': units, 'quantities': quantities}

Found 779 unique ingredients: ['salt', 'garlic', 'olive oil', 'sugar', 'butter']...
Found 52 unique units: ['cup', 'tablespoon', 'teaspoon', 'pound', 'ounce']...
Found 60 unique quantities: ['1', '2', '0', '1/2', '1/4']...


### Generating Positive Examples

In [8]:
import nltk
import random
from nltk.stem.wordnet import WordNetLemmatizer

random.seed(0)
lemmatizer = WordNetLemmatizer()

    
def generate_positive_instances(recipe_steps, max_new_actions=2):
    recipe_actions = []
    
    for recipe_step in recipe_steps:
        print('o-----------begin-----------o')
        print('Step:', recipe_step)
        step_actions = extract_step_actions(recipe_step)
        print('Actions:', create_phrases(step_actions))
        related_actions = search_related_actions(step_actions, max_new_actions)
        new_actions = create_phrases(related_actions)
        print('New Actions:', new_actions)
        recipe_actions.append({'step': recipe_step, 'actions': step_actions})
        print('o------------end------------o')
    

def extract_step_actions(recipe_step):
    sentences = nltk.sent_tokenize(recipe_step)
    step_actions = []
    for sentence in sentences:
        tokens, tags = tagger.predict_entities(sentence)
        sentence_actions = tagger.extract_action_relations(tokens, tags)
        step_actions += sentence_actions

    return step_actions


def search_related_actions(action_tuples, max_new_actions):
    related_actions = []
    
    for action, objects_in_action in action_tuples:
        action = convert_to_infinitive(action)
        if action in action_groups:
            all_related_actions = action_groups[action]
            if len(all_related_actions) > max_new_actions:
                all_related_actions = random.sample(all_related_actions, max_new_actions)
            selected_related_actions = [(a, objects_in_action) for a in all_related_actions]
            related_actions += selected_related_actions
        
    return related_actions


def convert_to_infinitive(verb):
    return lemmatizer.lemmatize(verb.lower(), 'v')

def create_phrases(action_tuples):
    phrases = []
    
    for action, objects_in_action in action_tuples:
        phrase = action.capitalize()
        if objects_in_action is not None:
            phrase += ' ' + objects_in_action
        phrases.append(phrase)
    
    return phrases
    

In [9]:
import json


def load_mit_recipe(recipe_id):
    with open('../../resource/recipes_oct_eval/recipe_%s.json' % recipe_id) as fin:
        recipe_data = json.load(fin)
        
    recipe_title = recipe_data['title']
    recipe_ingredients = [x for x in recipe_data['ingredients']]
    recipe_steps = [x for x in recipe_data['steps']]
    print('Recipe "%s" loaded!' % recipe_title)
    return (recipe_title, recipe_ingredients, recipe_steps)
    

In [10]:
recipe_title, recipe_ingredients, recipe_steps = load_mit_recipe('B')

Recipe "Pour-over Coffee" loaded!


In [11]:
generate_positive_instances(recipe_steps)

o-----------begin-----------o
Step: Measure 12 ounces of cold water and transfer to a kettle.
Actions: ['Measure water']
New Actions: ['Measure-in water', 'Measure-out water']
o------------end------------o
o-----------begin-----------o
Step: Assemble the filter cone.  Place the dripper on top of a coffee mug.
Actions: ['Assemble filter', 'Assemble cone', 'Place coffee', 'Place mug']
New Actions: ['Attach filter', 'Clip-on filter', 'Plug-into cone', 'Plug-in cone', 'Save coffee', 'Replace coffee', 'Combine-from mug', 'Return mug']
o------------end------------o
o-----------begin-----------o
Step: Prepare the filter insert by folding the paper filter in half to create a semi-circle, and in half again to create a quarter-circle. Place the paper filter in the dripper and spread open to create a cone.
Actions: ['Prepare', 'Folding', 'Create', 'Create', 'Place paper', 'Create']
New Actions: ['Prepare-for', 'Fold-up', 'Fold-into', 'Put-between', 'Place-on', 'Put-up', 'Place-on', 'Lie paper', '

### Generating Negative Examples

In [12]:
import random
random.seed(0)

def generate_negative_instances(recipe_sentences, entities_to_replace):
    create_wrong_recipe(recipe_sentences, entities_to_replace)
    
def create_wrong_recipe(recipe_sentences, entities_to_replace):
    recipe_entities = {'tokens': [], 'tags': []}
    
    for recipe_sentence in recipe_sentences:
        tokens, tags = tagger.predict_entities(recipe_sentence)
        recipe_entities['tokens'].append(list(tokens))
        recipe_entities['tags'].append(list(tags))
        
    new_recipe_entities = replace_entity(recipe_entities, entities_to_replace)
    print_changes(recipe_entities, new_recipe_entities, entities_to_replace)
    
def replace_entity(recipe_entities, entities_to_replace):
    new_recipe_entities = {'tokens': [], 'tags': recipe_entities['tags']}
    replacement_mapping = {}
    
    for sentence_tokens, sentence_tags in zip(recipe_entities['tokens'], recipe_entities['tags']):
        new_sentence_tokens = []
        for token, tag in zip(sentence_tokens, sentence_tags):
            if tag in entities_to_replace:
                if tag == 'UNIT':
                    new_token = select_random(token, corpus_recipe_entities['units'], replacement_mapping)
                elif tag == 'INGREDIENT':
                    new_token = select_random(token, corpus_recipe_entities['ingredients'], replacement_mapping)
                elif tag == 'QUANTITY':
                    new_token = select_random(token, corpus_recipe_entities['quantities'], replacement_mapping)
                replacement_mapping[token] = new_token
                token = new_token
            new_sentence_tokens.append(token)
            
        new_recipe_entities['tokens'].append(new_sentence_tokens)
    
    return new_recipe_entities
                
def select_random(token, options, replacement_mapping):
    if token in replacement_mapping:
        return replacement_mapping[token]
    
    return random.choice(options)
    
def print_recipe_text(recipe_title, recipe_entities):
    print('\n'.join([' '.join(x) for x in recipe_entities['tokens']]))

def print_recipe(recipe_title, recipe_entities, display_entities):
    for sentence_tokens, sentence_tags in zip(recipe_entities['tokens'], recipe_entities['tags']):
        tagger.plot_entities(sentence_tokens, sentence_tags, display_entities)
        
def print_changes(original_recipe_entities, new_recipe_entities, display_entities):
    for index in range(len(original_recipe_entities['tokens'])):
        sentence_tags = original_recipe_entities['tags'][index]
        sentence_tokens = original_recipe_entities['tokens'][index]
        new_sentence_tokens = new_recipe_entities['tokens'][index]
        print('o-----------begin-----------o')
        print('Original step:')
        tagger.plot_entities(sentence_tokens, sentence_tags, display_entities)
        print('Modified step:')
        tagger.plot_entities(new_sentence_tokens, sentence_tags, display_entities)
        print('o------------end------------o')

#### Replacing Ingredients

In [13]:
generate_negative_instances(recipe_steps, ['INGREDIENT'])

o-----------begin-----------o
Original step:


Modified step:


o------------end------------o
o-----------begin-----------o
Original step:


Modified step:


o------------end------------o
o-----------begin-----------o
Original step:


...

o------------end------------o
o-----------begin-----------o
Original step:


Modified step:


o------------end------------o
o-----------begin-----------o
Original step:


Modified step:


o------------end------------o


#### Replacing Quantities

In [14]:
generate_negative_instances(recipe_steps, ['QUANTITY'])

o-----------begin-----------o
Original step:


Modified step:


o------------end------------o
o-----------begin-----------o
Original step:


Modified step:


o------------end------------o
o-----------begin-----------o
Original step:


...

o------------end------------o
o-----------begin-----------o
Original step:


Modified step:


o------------end------------o
o-----------begin-----------o
Original step:


Modified step:


o------------end------------o


#### Replacing Units

In [15]:
generate_negative_instances(recipe_steps, ['UNIT'])

o-----------begin-----------o
Original step:


Modified step:


o------------end------------o
o-----------begin-----------o
Original step:


Modified step:


o------------end------------o
o-----------begin-----------o
Original step:


...

o------------end------------o
o-----------begin-----------o
Original step:


Modified step:


o------------end------------o
o-----------begin-----------o
Original step:


Modified step:


o------------end------------o
